encoder transformer层的linear2层（etl2）替换为近似矩阵乘法

In [1]:
import numpy as np
import os
import sys
dir_now = os.getcwd()
sys.path.append(dir_now)
sys.path.append(os.path.join(dir_now, '../'))
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE" # 防止jupyter爆内存
import matmul as mm
import math_util as mu
from NNutils import *
# import scipy.io as io
from amm_methods import *
import socket # Obtain the current host name, which can be used to select different data directories and result saving directories

In [2]:
method = METHOD_MITHRAL
# method = METHOD_PQ
# method = METHOD_PLUTO
# method = METHOD_MITHRALPQ
# method = METHOD_EXACT
# method = METHOD_SCALAR_QUANTIZE
quantize_lut = True

In [3]:
linear_name = 'etl2'
feedback_bits = 256
linear_name_full = "ex_linear2"

auto_train = False # 是否根据已运行的训练性能结果自动训练，（train_sam_num取已训练的最大值）

nbits_trained = 8
nbits_goal = 8
if quantize_lut == False:
    nbits_goal = 0
nbits = nbits_goal # 要运行的量化比特数

test_sam_num = 1000 # 测试集样本数(如需修改，请同时修改下面的读取文件，现文件默认1000个样本)

if not auto_train:
    ncodebooks = 64 # max:512
    ncentroids = 16
    train_sam_num = 100 # 训练集样本数
else:
    cb_ct_ntr_combinations_unique = change_nbits_auto_run_list(linear_name, method, feedback_bits, nbits_trained, nbits_goal)
    print(cb_ct_ntr_combinations_unique)
    # 遍历每个cb、ct、n_train_sam组合
    # for _, row_ref in cb_ct_ntr_combinations_unique.iterrows():
    #     ncodebooks = int(row_ref['cb'])
        # ncentroids = int(row_ref['ct'])
        # train_sam_num = int(row_ref['n_train_sam'])

batch_size = 32
if method == METHOD_EXACT:
    ncodebooks = 0
    ncentroids = 0

In [4]:
AMM_train_dirs = get_AMM_train_dirs(linear_name, linear_name_full, method, feedback_bits, train_sam_num, test_sam_num)
create_dir(AMM_train_dirs["dir_result"])

# host_name = socket.gethostname()
# if host_name == 'DESKTOP-PLRL7TK':
#     dir_train = 'E:\\hdr\\研一\\华为-深度学习\\intermediate\\intermediate8dbfc1'
#     dir_result = ''
# elif host_name == 'DESKTOP-6FOH47P':
#     dir_train = 'F:\\Projects\\python\\PQ\\intermediate8dbfc1'
#     dir_result = 'F:\\Projects\\python\\PQ\\res'
#     linearin_path_train= ''
#     linearout_path_train= ''
#     linearin_path_test = ''
#     linearout_path_test = ''
# elif host_name == 'jm-System-Product-Name':
#     dir_joined = '/data/hdr/transformer_data/joined'
#     dir_train = os.path.join(dir_joined, 'train', 'f'+str(feedback_bits))
#     dir_test = os.path.join(dir_joined, 'test', 'f'+str(feedback_bits))
#     dir_result = '/data/hdr/pq/res'
#     linearin_path_train= '%sin_train_f%i_sam%i.npy' % (linear_name_full, feedback_bits, train_sam_num)
#     y_train = '%s_y_train_f%i_sam%i.npy' % (linear_name_full, feedback_bits, train_sam_num)
#     linearout_path_train= '%sout_train_f%i_sam%i.npy' % (linear_name_full, feedback_bits, train_sam_num)
#     linearin_path_test = '%sin_test_f%i_sam%i.npy' % (linear_name_full, feedback_bits, test_sam_num)
#     linearout_path_test = '%sout_test_f%i_sam%i.npy' % (linear_name_full, feedback_bits, test_sam_num)
# else:
#     raise NameError("You are running the script in a new computer %s, please define dirs" % host_name)


# weightpath = '%s_w_f%i.npy' % (linear_name_full, feedback_bits)
# biaspath = '%s_b_f%i.npy' % (linear_name_full, feedback_bits)
# dir_result = os.path.join(dir_result, method, "f%i" % feedback_bits, linear_name)
# try:
#     os.mkdir(dir_result)
# except FileNotFoundError:
#     os.makedirs(dir_result)
# except FileExistsError:
#     pass 


In [5]:
dataset_prepare(AMM_train_dirs["dir_joined"], linear_name_full, feedback_bits, [train_sam_num, test_sam_num], 
                batch_size, S1 = S1_dict[linear_name])

In [6]:
if method == METHOD_PLUTO:
    est3 = mm.estFactory(X_path=AMM_train_dirs["linearin_path_train"], W_path=AMM_train_dirs["weightpath"], 
                        Y_path=AMM_train_dirs["y_train"], dir= AMM_train_dirs["dir_train"], ncodebooks=ncodebooks, 
                        ncentroids=ncentroids, methods=[method], nbits=nbits, quantize_lut = quantize_lut, 
                        bias_path=AMM_train_dirs["biaspath"])
else:
    est3 = mm.estFactory(X_path=AMM_train_dirs["linearin_path_train"], W_path=AMM_train_dirs["weightpath"], 
                        Y_path=AMM_train_dirs["y_train"], dir= AMM_train_dirs["dir_train"], ncodebooks=ncodebooks, 
                        ncentroids=ncentroids, methods=[method], nbits=nbits, quantize_lut = quantize_lut)

running method:  Mithral
X.shape:  (102400, 512)
_learn_mithral_initialization heuristic pq
learn_multisplits(): initial loss:    12067.76677696843
learn_multisplits(): returning loss:  248.2884956598282
learn_multisplits(): initial loss:    437.69855046365154
learn_multisplits(): returning loss:  115.66391400824564
learn_multisplits(): initial loss:    7288.489180909765
learn_multisplits(): returning loss:  137.677903175354
learn_multisplits(): initial loss:    11723.823941795165
learn_multisplits(): returning loss:  638.7195053100586
learn_multisplits(): initial loss:    132.81804854364083
learn_multisplits(): returning loss:  36.05963219350335
learn_multisplits(): initial loss:    66.36084221527102
learn_multisplits(): returning loss:  16.74364847965531
learn_multisplits(): initial loss:    238.83692935144026
learn_multisplits(): returning loss:  61.94578514468796
learn_multisplits(): initial loss:    6507.092761617811
learn_multisplits(): returning loss:  169.4231570661068
learn_mu

In [7]:
x_test = np.load(AMM_train_dirs["dir_test"]+'/'+AMM_train_dirs["linearin_path_test"])
w_test = np.load(AMM_train_dirs["dir_train"]+'/'+AMM_train_dirs["weightpath"])
bias = np.load(AMM_train_dirs["dir_train"]+'/'+AMM_train_dirs["biaspath"])
# print(type(est3))
y_out_matmul = mm.eval_matmul(est3, x_test, w_test) # MADDNESS乘法的结果
# y_out_last = mu.softmax(y_out_matmul + bias.T) # MADDNESS替换后当前层输出，即+bias并激活函数后的结果
if method == METHOD_PLUTO:
    y_out_last = y_out_matmul
else:
    y_out_last = y_out_matmul + bias.T # MADDNESS替换后当前层输出，即+bias并不需要激活函数后的结果

刚好在生成lut的代码前的质心：
 [[[ 7.10644480e-03  6.49576832e-04 -4.42077406e-04 ...  8.54550512e-04
   -5.33470511e-03 -1.75292254e-03]
  [ 2.97305849e-03  1.50031003e-03  1.19184726e-04 ...  8.55438644e-04
    7.68309878e-03 -4.58348542e-03]
  [ 3.39981681e-03  5.95193887e-05 -1.79810566e-04 ...  8.27314507e-04
   -5.00277989e-03 -1.00134080e-03]
  ...
  [-2.00153281e-05 -7.94074091e-04 -9.65518993e-05 ... -4.21199540e-04
    1.41145277e-03  8.80998792e-04]
  [-6.89144304e-04 -8.30574660e-04 -1.53885616e-04 ... -3.76614655e-04
   -1.14855357e-03  1.98233858e-04]
  [ 1.55301168e-05 -8.26850301e-04 -1.45828730e-04 ... -2.61677953e-04
   -3.77492979e-04  6.42693252e-04]]

 [[-3.21029656e-04 -2.33380735e-04  3.12766526e-04 ... -6.34681710e-05
   -2.66890856e-03  2.20365240e-04]
  [ 1.69224525e-03 -9.56519798e-04 -1.22328813e-03 ... -2.54605518e-04
    4.70361626e-03 -2.44945777e-03]
  [ 1.55133210e-04  7.28597282e-04  4.65999852e-04 ... -2.74449878e-04
   -2.53985520e-03  1.44615513e-03]
  ...
  [ 0

In [8]:
print(y_out_last)
print("y_out_last.shape: ", y_out_last.shape)
y_out_last_re = y_out_last.reshape(test_sam_num, batch_size, -1, y_out_last.shape[-1]) #AMM字典模式需要复原y大小
print("y_out_last_re.shape: ", y_out_last_re.shape)
if method == METHOD_SCALAR_QUANTIZE:
    np.save(os.path.join(AMM_train_dirs["dir_result"], '%s%s_trsam%i_tesam%i_fb%i_nbits%i.npy' % 
                                                        (method, linear_name, train_sam_num, test_sam_num, feedback_bits, nbits)), 
                                                        y_out_last_re.astype(np.float32))
elif method == METHOD_MITHRAL or method == METHOD_PQ or method == METHOD_PLUTO or method == METHOD_MITHRALPQ:
    np.save(os.path.join(AMM_train_dirs["dir_result"], '%s%s_ql%i_nbits%i_trsam%i_tesam%i_fb%i_cb%i_ct%i.npy' % 
                                                        (method, linear_name, quantize_lut, nbits, train_sam_num, test_sam_num, 
                                                        feedback_bits, ncodebooks, ncentroids)), y_out_last_re)
else:
    np.save(os.path.join(AMM_train_dirs["dir_result"], '%s%s_trsam%i_tesam%i_fb%i_cb%i_ct%i.npy' % 
                                                        (method, linear_name, train_sam_num, test_sam_num, feedback_bits, 
                                                        ncodebooks, ncentroids)), y_out_last_re)

[[ 0.21000968 -0.3227942   0.6800591  ...  0.19014056  0.00927814
   0.38835266]
 [ 0.21000968 -0.07230497  0.6800591  ...  0.19014056  0.00927814
   0.38835266]
 [ 0.21000968 -0.3227942   0.6800591  ...  0.19014056  0.00927814
   0.63884187]
 ...
 [ 0.21000968  0.17818426  0.42956984 ...  0.19014056  0.25976738
   0.1378634 ]
 [ 0.46049893 -0.3227942   0.6800591  ... -0.06034867  0.00927814
   0.38835266]
 [ 0.46049893  0.17818426  0.42956984 ...  0.19014056  0.25976738
   0.1378634 ]]
y_out_last.shape:  (1024000, 64)
y_out_last_re.shape:  (1000, 32, 32, 64)
